In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import KernelPCA

path = "/home/ubuntu/onekgenomes/"
sampleDataFile = "data/sampleData/sampleData.tsv"
df = pd.read_csv(path + sampleDataFile, sep='\t', index_col=0)

In [ ]:
pops = df["Population"].tolist()
uniquePops = list(set(pops))
uniquePops.sort()
numSamples = len(pops)
targetDict = dict(zip(uniquePops, range(numSamples)))
Y = np.array([targetDict[pop] for pop in pops])

In [ ]:
superPops = df["Super Population"].tolist()
uniquePops = list(set(superPops))
uniquePops.sort()
numSamples = len(superPops)
targetDict = dict(zip(uniquePops, range(numSamples)))
Y = np.array([targetDict[p] for p in superPops])

In [ ]:
genders = df["Gender"]
G = (genders == "male").values

In [ ]:
pdistMat = np.load(path + "data/pdist/summedMats/pdistAll.npy")
sigmaSq = np.quantile(pdistMat.reshape(-1), 0.1)
gamma = 1 / (2 * sigmaSq)
K_all = np.exp(-gamma * pdistMat)
numComp = 4
embedding = KernelPCA(kernel="precomputed", n_components=numComp)
embeddedData = embedding.fit_transform(K_all)
embeddedData.shape
x, y, z, w = np.hsplit(embeddedData, numComp)

In [ ]:
pdistMat = np.load(path + "data/pdist/summedMats/pdistNum.npy")
sigmaSq = np.quantile(pdistMat.reshape(-1), 0.1)
gamma = 1 / (2 * sigmaSq)
K_num = np.exp(-gamma * pdistMat)
numComp = 4
embedding = KernelPCA(kernel="precomputed", n_components=numComp)
embeddedData = embedding.fit_transform(K_num)
embeddedData.shape
x1, y1, z1, w1 = np.hsplit(embeddedData, numComp)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax1 = fig.add_subplot(221, projection="3d")
cmap = cm.rainbow(Y / np.max(Y))
ax1.scatter(x, y, z, c=cmap)
ax2 = fig.add_subplot(222, projection="3d")
genderCmap = np.zeros((numSamples,))
genderCmap[G] = 1
ax2.scatter(x, y, z, c=genderCmap)
ax3 = fig.add_subplot(223, projection="3d")
ax3.scatter(x1, y1, z1, c=cmap)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig2 = plt.figure()
ax4 = fig2.add_subplot(121)
cmap = cm.rainbow(Y / np.max(Y))
ax4.scatter(y,z, c=cmap)
ax4.set_xlim(-0.0015,0.0015)
ax4.set_ylim(-0.0015,0.0015)
ax5 = fig2.add_subplot(122)
ax5.scatter(x1, y1, c=cmap)
ax5.set_xlim(-0.0015,0.0015)
ax5.set_ylim(-0.0015,0.0015)

In [ ]:
import numpy as np
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import KernelCenterer
from numpy.linalg import svd

In [6]:
path = "/home/ubuntu/onekgenomes/"
num_comp = 3
D_sq = np.load(path + "data/pdist/summedMats/pdistNum.npy")
m = D_sq.shape[0]
sigma_sq = np.quantile(D_sq.reshape(-1), 0.5)
gamma = 1 / (2 * sigma_sq)
K = np.exp(-gamma * D_sq)
centerer = KernelCenterer().fit(K)
K_c1 = centerer.transform(K)
K_c2 = (np.eye(m) - 1 / m) @ K @ (np.eye(m) - 1 / m)
np.max(np.abs(K_c1 - K_c2))

4.1147640850169864e-15

In [7]:
import numpy as np
from numpy.linalg import svd

def rbf_kernel_pca(D_sq, gamma, num_comp):
    m = D_sq.shape[0]
    K = np.exp(-gamma * D_sq)
    K_c = (np.eye(m) - 1 / m) @ K @ (np.eye(m) - 1 / m)
    U, Sigma_sq, _ = svd(K_c)
    Sigma = np.sqrt(Sigma_sq[:num_comp])
    return U[:,:num_comp] * Sigma

In [8]:
path = "/home/ubuntu/onekgenomes/"
num_comp = 3
D_sq = np.load(path + "data/pdist/summedMats/pdistAll.npy")
sigma_sq = np.quantile(D_sq.reshape(-1), 0.5)
gamma = 1 / (2 * sigma_sq)
temp = rbf_kernel_pca(D_sq, gamma, num_comp)
saveFile = path + "data/dimReduc/kernelPCA/manualEmbeddedAll.npy"
np.save(saveFile, temp)

In [9]:
path = "/home/ubuntu/onekgenomes/"
num_comp = 3
D_sq = np.load(path + "data/pdist/summedMats/pdistNum.npy")
sigma_sq = np.quantile(D_sq.reshape(-1), 0.5)
gamma = 1 / (2 * sigma_sq)
temp = rbf_kernel_pca(D_sq, gamma, num_comp)
saveFile = path + "data/dimReduc/kernelPCA/manualEmbeddedNum.npy"
np.save(saveFile, temp)

In [ ]:
path = "/home/ubuntu/onekgenomes/"
num_comp = 3
D_sq = np.load(path + "data/pdist/summedMats/pdistAll.npy")
sigma_sq = np.quantile(pdistMat.reshape(-1), 0.5)
gamma = 1 / (2 * sigma_sq)
K = np.exp(-gamma * D_sq)
centerer = KernelCenterer().fit(K)
K_c = centerer.transform(K)
U, Sigma_sq, _ = svd(K_c)
Sigma = np.sqrt(Sigma_sq[:num_comp])
temp = U[:,:num_comp] * Sigma
saveFile = path + "data/dimReduc/kernelPCA/manualEmbeddedAll.npy"
np.save(saveFile, temp)

In [ ]:
path = "/home/ubuntu/onekgenomes/"
num_comp = 3
D_sq = np.load(path + "data/pdist/summedMats/pdistNum.npy")
sigma_sq = np.quantile(pdistMat.reshape(-1), 0.5)
gamma = 1 / (2 * sigma_sq)
K = np.exp(-gamma * D_sq)
centerer = KernelCenterer().fit(K)
K_c = centerer.transform(K)
U, Sigma_sq, _ = svd(K_c)
Sigma = np.sqrt(Sigma_sq[:num_comp])
temp = U[:,:num_comp] * Sigma
saveFile = path + "data/dimReduc/kernelPCA/manualEmbeddedNum.npy"
np.save(saveFile, temp)

In [ ]:
path = "/home/ubuntu/onekgenomes/"
pdistMat = np.load(path + "data/pdist/summedMats/pdistAll.npy")
sigmaSq = np.quantile(pdistMat.reshape(-1), 0.5)
gamma = 1 / (2 * sigmaSq)
K = np.exp(-gamma * pdistMat)
centerer = KernelCenterer().fit(K)
K = centerer.transform(K)
numComp = 3
embedding = KernelPCA(kernel="precomputed", n_components=numComp)
embedded = embedding.fit_transform(K)
saveFile = path + "data/dimReduc/kernelPCA/embeddedAll.npy"
#np.save(saveFile, embedded)

In [ ]:
np.mean(abs(U[:,:num_comp] - embedded))

In [ ]:
pdistMat = np.load(path + "data/pdist/summedMats/pdistNum.npy")
sigmaSq = np.quantile(pdistMat.reshape(-1), 0.5)
gamma = 1 / (2 * sigmaSq)
K = np.exp(-gamma * pdistMat)
numComp = 3
embedding = KernelPCA(kernel="precomputed", n_components=numComp)
embedded = embedding.fit_transform(K)
saveFile = path + "data/dimReduc/kernelPCA/embeddedNum.npy"
np.save(saveFile, embedded)